**FOR FASTER COMPUTATION**

In [1]:
!pip install pymupdf
!pip install langchain_community
!pip install langchain
!pip install faiss-cpu  # faiss-gpu might crash if no GPU is present
!pip install transformers
!pip install sentence-transformers
!pip install scikit-learn


import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")
from transformers import AutoModelForSeq2SeqLM


# Load document
loader = PyMuPDFLoader("/kaggle/input/booooook/Harry Potter Book.pdf")
pages = loader.load_and_split()

# Configurable lists
embedding_models = [
    "sentence-transformers/all-mpnet-base-v2",                # High-quality general-purpose embeddings
    "sentence-transformers/multi-qa-mpnet-base-dot-v1",       # Optimized for QA retrieval
    "sentence-transformers/paraphrase-MiniLM-L6-v2",          # Efficient, high-quality paraphrase embeddings
    "sentence-transformers/all-distilroberta-v1"              # Distilled version of RoBERTa, fast and effective
]

generative_models = [
    "google/flan-t5-base",             # Flan-T5 base, a highly tuned model for general tasks
    "facebook/bart-large-cnn",         # BART, known for great performance in text generation and summarization
    "mistral-7b",                      # A smaller but powerful open-weight model for complex tasks
    "t5-base"                          # Balanced and general-purpose, smaller than T5-large, but still effective
]

# Utility functions
def get_embeddings_for_documents(documents, model):
    return model.embed_documents(documents)

def get_embedding_for_query(query, model):
    return model.embed_query(query)

# Parameters
chunk_size = 1000
chunk_overlap = 750
k = 3
question = "Who was Hagrid?"
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
chunks = text_splitter.split_documents(pages)

# Store results
results = []

# Loop through combinations
for embed_model_name in embedding_models:
    embedding_model = HuggingFaceEmbeddings(model_name=embed_model_name)
    vectordb = FAISS.from_documents(chunks, embedding_model)
    retriever = vectordb.as_retriever(search_kwargs={"k": k})
    docs = retriever.get_relevant_documents(question)

    retrieved_documents = [doc.page_content for doc in docs[:k]]
    retrieved_responses = [f"Page {doc.metadata.get('page', 'Unknown')}: {doc.page_content.strip()}" for doc in docs]
    while len(retrieved_responses) < 5:
        retrieved_responses.append("No relevant information.")

    for gen_model_name in generative_models:
        try:
            model = AutoModelForSeq2SeqLM.from_pretrained(gen_model_name, device_map="auto", torch_dtype="auto", trust_remote_code=True)
            tokenizer = AutoTokenizer.from_pretrained(gen_model_name)
            generator = pipeline("text-generation", model=model, tokenizer=tokenizer, return_full_text=False, max_new_tokens=300)

            prompt = f"""
You are an AI assistant tasked with answering questions based on retrieved knowledge.

### **Retrieved Information**:
1️⃣ {retrieved_responses[0]}
2️⃣ {retrieved_responses[1]}
3️⃣ {retrieved_responses[2]}

### **Question**:
{question}

### **Instructions**:
- Integrate the key points from all retrieved responses into a **cohesive, well-structured answer**.
- If the responses are **contradictory**, mention the different perspectives.
- If none of the retrieved responses contain relevant information, reply:
  **"I couldn't find a good response to your query in the database."**
"""

            output = generator(prompt)[0]["generated_text"]
            generated_embedding = get_embedding_for_query(output, embedding_model)
            doc_embeddings = get_embeddings_for_documents(retrieved_documents, embedding_model)

            # Cosine similarities
            similarities = [cosine_similarity([doc_emb], [generated_embedding])[0][0] for doc_emb in doc_embeddings]

            results.append({
                "Embedding Model": embed_model_name,
                "Generative Model": gen_model_name,
                "Faithfulness": round(max(similarities), 4),
                "Relevancy": round(np.mean(similarities), 4),
            })
        except Exception as e:
            print(f"❌ Error using {embed_model_name} + {gen_model_name}: {e}")
            continue

# Final summary
print("\n\n=== Summary of Results ===")
print(f"{'Embedding Model':<45} {'Generative Model':<30} {'Faithfulness':<15} {'Relevancy'}")
for r in results:
    print(f"{r['Embedding Model']:<45} {r['Generative Model']:<30} {r['Faithfulness']:<15} {r['Relevancy']}")



config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Device set to use cuda:0
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'Mar

❌ Error using sentence-transformers/all-distilroberta-v1 + facebook/bart-large-cnn: weight is on the meta device, we need a `value` to put in on 0.
❌ Error using sentence-transformers/all-distilroberta-v1 + mistral-7b: mistral-7b is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`


Device set to use cuda:0
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'Mar



=== Summary of Results ===
Embedding Model                               Generative Model               Faithfulness    Relevancy
sentence-transformers/all-mpnet-base-v2       google/flan-t5-base            0.0648          0.0563
sentence-transformers/all-mpnet-base-v2       t5-base                        0.0648          0.0563
sentence-transformers/multi-qa-mpnet-base-dot-v1 google/flan-t5-base            0.228           0.2006
sentence-transformers/multi-qa-mpnet-base-dot-v1 t5-base                        0.228           0.2006
sentence-transformers/paraphrase-MiniLM-L6-v2 google/flan-t5-base            0.168           0.1521
sentence-transformers/paraphrase-MiniLM-L6-v2 t5-base                        0.168           0.1521
sentence-transformers/all-distilroberta-v1    google/flan-t5-base            0.1333          0.1241
sentence-transformers/all-distilroberta-v1    t5-base                        0.1333          0.1241


In [2]:
import pandas as pd

# Save results to a CSV
results_df = pd.DataFrame(results)
results_df.to_csv("rag_results.csv", index=False)

print("\n✅ Results saved to 'rag_results.csv'")


✅ Results saved to 'rag_results.csv'
